In [9]:
import pandas as pd

In [10]:
words = pd.DataFrame(['apple','capybara','carburetor','penapple'],columns = ['words'])

In [11]:
words

,words
0,apple
1,capybara
2,carburetor
3,penapple


In [12]:
pd.get_dummies(words)

,words_apple,words_capybara,words_carburetor,words_penapple
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1


In [13]:
from gensim.models import Word2Vec

In [14]:
model = Word2Vec(vector_size = 500, window=10, min_count=2, sg=1)

In [15]:
f = open("/content/Лев Толстой. Война и мир.txt",'r',encoding="utf-8")

In [16]:
text = f.readlines()

In [17]:
text = ' '.join(text)

In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('corpus')

In [20]:
import nltk
from nltk.tokenize import TweetTokenizer, sent_tokenize

tokenizer_words = TweetTokenizer()
tokens_sentences = [tokenizer_words.tokenize(t) for t in
nltk.sent_tokenize(text)]

In [21]:
model.build_vocab(tokens_sentences)

In [22]:
tokens_sentences[5]

['(',
 'В',
 'дальнейшем',
 'переводы',
 'с',
 'французского',
 'не',
 'оговариваются',
 '.']

In [23]:
model.train(tokens_sentences, total_examples=model.corpus_count, epochs=30, report_delay=1)

(6140280, 9113910)

In [24]:
model.wv.most_similar('император')

[('Франц', 0.6664320230484009),
 ('Александр', 0.6510893106460571),
 ('Виват', 0.5873460173606873),
 ('изменит', 0.5690732598304749),
 ('согласится', 0.564491868019104),
 ('российский', 0.5631046891212463),
 ('354', 0.5575778484344482),
 ('смешался', 0.5569444894790649),
 ('упомянутое', 0.5551967620849609),
 ('возвратить', 0.5450008511543274)]

In [25]:
model.wv.most_similar('француз')

[('горбатым', 0.617556631565094),
 ('красавец', 0.5911628007888794),
 ('ружье', 0.5743246078491211),
 ('банник', 0.5528762340545654),
 ('слоге', 0.5467669367790222),
 ('добежать', 0.5239396691322327),
 ('vieux', 0.5162952542304993),
 ('артиллериста', 0.5155836343765259),
 ('Французский', 0.5154901742935181),
 ('174', 0.5092029571533203)]

In [ ]:
import gensim
model2 = gensim.models.KeyedVectors.load_word2vec_format("/content/drive/MyDrive/emb/ruwikiruscorpora-nobigrams_upos_skipgram_300_5_2018.vec.gz")
model2.init_sims(replace=True)

In [27]:
model2.most_similar('император_NOUN')

[('император_PROPN', 0.8153896331787109),
 ('императрица_NOUN', 0.7798320055007935),
 ('му-цзун_PROPN', 0.7033633589744568),
 ('дай-цзун_PROPN', 0.6861872673034668),
 ('тай-цзун_PROPN', 0.6790136098861694),
 ('чжао-цзун_PROPN', 0.6768495440483093),
 ('тэндзи_PROPN', 0.6718018054962158),
 ('гао-цзун_PROPN', 0.6703536510467529),
 ('цезаирать_NOUN', 0.6679410934448242),
 ('экс-император_NOUN', 0.6674522161483765)]

In [28]:
model2.most_similar('француз_NOUN')

[('англичанин_NOUN', 0.8143453001976013),
 ('австриец_NOUN', 0.7257215976715088),
 ('итальянец_NOUN', 0.7058789134025574),
 ('пруссак_NOUN', 0.705450177192688),
 ('испанец_NOUN', 0.6938128471374512),
 ('турок_NOUN', 0.6631002426147461),
 ('пруссаки_NOUN', 0.6606427431106567),
 ('лотарингец_NOUN', 0.660156786441803),
 ('голландец_NOUN', 0.6587073802947998),
 ('швейцарец_NOUN', 0.656599760055542)]

Тестирование работы на реальных данных:

In [29]:
text_from_telegram = """ЕС борется с микропластиком
 В конце сентября Еврокомиссия приняла решение о запрете изделий, при производстве которых используется микропластик, то есть частицы пластика размером менее 5 миллиметров в длину. Речь о продукции в которую пластиковые частицы добавляют намеренно, в результате чего, при нормальном использовании они становятся мощным источником загрязнения микропластиком.

Процесс этот постепенный, первой в ЕС, в рамках цели по сокращению загрязнения микропластиком на 30% к 2030 году, 15 октября запретили косметику с добавлением микропластика, например, блёстки. Это вызвало волну спроса на уже произведённые товары. Изготовителям других продуктов предоставят период отсрочки, чтобы они могли приспособиться к запрету и найти альтернативы, в некоторых случаях ограничения могут вступить в силу только через несколько лет.

Ежегодно только в Евросоюзе в окружающую среду выбрасывается от 52 до 184 тысяч тонн пластиковых частиц. Большую часть из этого можно относительно просто предотвратить – отказавшись от использования.

Если ничего не делать, то, как следует из отчёта некоммерческой организации Pew Charitable Trusts, к 2040 году глобальное загрязнение микропластиком удвоится."""

In [ ]:
!pip install wget

In [ ]:
!pip install ufal.udpipe

In [32]:
#!/usr/bin/env python3
# coding: utf-8

import sys
import os
import wget
import re
from ufal.udpipe import Model, Pipeline

"""
Этот скрипт принимает на вход необработанный русский текст
(одно предложение на строку или один абзац на строку).
Он токенизируется, лемматизируется и размечается по частям речи с использованием UDPipe.
На выход подаётся последовательность разделенных пробелами лемм с частями речи
("зеленый_ADJ трамвай_NOUN").
Их можно непосредственно использовать в моделях с RusVectōrēs (https://rusvectores.org).

Примеры запуска:
echo 'Мама мыла раму.' | python3 rus_preprocessing_udpipe.py
zcat large_corpus.txt.gz | python3 rus_preprocessing_udpipe.py | gzip > processed_corpus.txt.gz
"""


def num_replace(word):
    newtoken = "x" * len(word)
    return newtoken


def clean_token(token, misc):
    """
    :param token:  токен (строка)
    :param misc:  содержимое поля "MISC" в CONLLU (строка)
    :return: очищенный токен (строка)
    """
    out_token = token.strip().replace(" ", "")
    if token == "Файл" and "SpaceAfter=No" in misc:
        return None
    return out_token


def clean_lemma(lemma, pos):
    """
    :param lemma: лемма (строка)
    :param pos: часть речи (строка)
    :return: очищенная лемма (строка)
    """
    out_lemma = lemma.strip().replace(" ", "").replace("_", "").lower()
    if "|" in out_lemma or out_lemma.endswith(".jpg") or out_lemma.endswith(".png"):
        return None
    if pos != "PUNCT":
        if out_lemma.startswith("«") or out_lemma.startswith("»"):
            out_lemma = "".join(out_lemma[1:])
        if out_lemma.endswith("«") or out_lemma.endswith("»"):
            out_lemma = "".join(out_lemma[:-1])
        if (
            out_lemma.endswith("!")
            or out_lemma.endswith("?")
            or out_lemma.endswith(",")
            or out_lemma.endswith(".")
        ):
            out_lemma = "".join(out_lemma[:-1])
    return out_lemma


def list_replace(search, replacement, text):
    search = [el for el in search if el in text]
    for c in search:
        text = text.replace(c, replacement)
    return text


def unify_sym(text):  # принимает строку в юникоде
    text = list_replace(
        "\u00AB\u00BB\u2039\u203A\u201E\u201A\u201C\u201F\u2018\u201B\u201D\u2019",
        "\u0022",
        text,
    )

    text = list_replace(
        "\u2012\u2013\u2014\u2015\u203E\u0305\u00AF", "\u2003\u002D\u002D\u2003", text
    )

    text = list_replace("\u2010\u2011", "\u002D", text)

    text = list_replace(
        "\u2000\u2001\u2002\u2004\u2005\u2006\u2007\u2008\u2009\u200A\u200B\u202F\u205F\u2060\u3000",
        "\u2002",
        text,
    )

    text = re.sub("\u2003\u2003", "\u2003", text)
    text = re.sub("\t\t", "\t", text)

    text = list_replace(
        "\u02CC\u0307\u0323\u2022\u2023\u2043\u204C\u204D\u2219\u25E6\u00B7\u00D7\u22C5\u2219\u2062",
        ".",
        text,
    )

    text = list_replace("\u2217", "\u002A", text)

    text = list_replace("…", "...", text)

    text = list_replace("\u2241\u224B\u2E2F\u0483", "\u223D", text)

    text = list_replace("\u00C4", "A", text)  # латинская
    text = list_replace("\u00E4", "a", text)
    text = list_replace("\u00CB", "E", text)
    text = list_replace("\u00EB", "e", text)
    text = list_replace("\u1E26", "H", text)
    text = list_replace("\u1E27", "h", text)
    text = list_replace("\u00CF", "I", text)
    text = list_replace("\u00EF", "i", text)
    text = list_replace("\u00D6", "O", text)
    text = list_replace("\u00F6", "o", text)
    text = list_replace("\u00DC", "U", text)
    text = list_replace("\u00FC", "u", text)
    text = list_replace("\u0178", "Y", text)
    text = list_replace("\u00FF", "y", text)
    text = list_replace("\u00DF", "s", text)
    text = list_replace("\u1E9E", "S", text)

    currencies = list(
        "\u20BD\u0024\u00A3\u20A4\u20AC\u20AA\u2133\u20BE\u00A2\u058F\u0BF9\u20BC\u20A1\u20A0\u20B4\u20A7\u20B0\u20BF\u20A3\u060B\u0E3F\u20A9\u20B4\u20B2\u0192\u20AB\u00A5\u20AD\u20A1\u20BA\u20A6\u20B1\uFDFC\u17DB\u20B9\u20A8\u20B5\u09F3\u20B8\u20AE\u0192"
    )

    alphabet = list(
        '\t\n\r абвгдеёзжийклмнопрстуфхцчшщьыъэюяАБВГДЕЁЗЖИЙКЛМНОПРСТУФХЦЧШЩЬЫЪЭЮЯ,.[]{}()=+-−*&^%$#@!?~;:0123456789§/\|"abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ '
    )

    alphabet.append("'")

    allowed = set(currencies + alphabet)

    cleaned_text = [sym for sym in text if sym in allowed]
    cleaned_text = "".join(cleaned_text)

    return cleaned_text


def process(pipeline, text="Строка", keep_pos=True, keep_punct=False):
    # Если частеречные тэги не нужны (например, их нет в модели), выставьте pos=False
    # в этом случае на выход будут поданы только леммы
    # По умолчанию знаки пунктуации вырезаются. Чтобы сохранить их, выставьте punct=True

    entities = {"PROPN"}
    named = False
    memory = []
    mem_case = None
    mem_number = None
    tagged_propn = []

    # обрабатываем текст, получаем результат в формате conllu:
    processed = pipeline.process(text)

    # пропускаем строки со служебной информацией:
    content = [line for line in processed.split("\n") if not line.startswith("#")]

    # извлекаем из обработанного текста леммы, тэги и морфологические характеристики
    tagged = [w.split("\t") for w in content if w]

    for t in tagged:
        if len(t) != 10:
            continue
        (word_id, token, lemma, pos, xpos, feats, head, deprel, deps, misc) = t
        token = clean_token(token, misc)
        lemma = clean_lemma(lemma, pos)
        if not lemma or not token:
            continue
        if pos in entities:
            if "|" not in feats:
                tagged_propn.append("%s_%s" % (lemma, pos))
                continue
            morph = {el.split("=")[0]: el.split("=")[1] for el in feats.split("|")}
            if "Case" not in morph or "Number" not in morph:
                tagged_propn.append("%s_%s" % (lemma, pos))
                continue
            if not named:
                named = True
                mem_case = morph["Case"]
                mem_number = morph["Number"]
            if morph["Case"] == mem_case and morph["Number"] == mem_number:
                memory.append(lemma)
                if "SpacesAfter=\\n" in misc or "SpacesAfter=\s\\n" in misc:
                    named = False
                    past_lemma = "::".join(memory)
                    memory = []
                    tagged_propn.append(past_lemma + "_PROPN")
            else:
                named = False
                past_lemma = "::".join(memory)
                memory = []
                tagged_propn.append(past_lemma + "_PROPN")
                tagged_propn.append("%s_%s" % (lemma, pos))
        else:
            if not named:
                if (
                    pos == "NUM" and token.isdigit()
                ):  # Заменяем числа на xxxxx той же длины
                    lemma = num_replace(token)
                tagged_propn.append("%s_%s" % (lemma, pos))
            else:
                named = False
                past_lemma = "::".join(memory)
                memory = []
                tagged_propn.append(past_lemma + "_PROPN")
                tagged_propn.append("%s_%s" % (lemma, pos))

    if not keep_punct:
        tagged_propn = [word for word in tagged_propn if word.split("_")[1] != "PUNCT"]
    if not keep_pos:
        tagged_propn = [word.split("_")[0] for word in tagged_propn]
    return tagged_propn


# URL of the UDPipe model
udpipe_model_url = "https://rusvectores.org/static/models/udpipe_syntagrus.model"
udpipe_filename = udpipe_model_url.split("/")[-1]

if not os.path.isfile(udpipe_filename):
    print("UDPipe model not found. Downloading...", file=sys.stderr)
    wget.download(udpipe_model_url)

print("\nLoading the model...", file=sys.stderr)
model = Model.load(udpipe_filename)
process_pipeline = Pipeline(
    model, "tokenize", Pipeline.DEFAULT, Pipeline.DEFAULT, "conllu"
)

print("Processing input...", file=sys.stderr)
for input_line in sys.stdin:
    res = unify_sym(input_line.strip())
    output = process(process_pipeline, text=res)
    print(" ".join(output))

from ufal.udpipe import Model, Pipeline
import os
import re
import sys

def tag_ud(text='Текст нужно передать функции в виде строки!', modelfile='udpipe_syntagrus.model'):
    udpipe_model_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
    udpipe_filename = udpipe_model_url.split('/')[-1]

    if not os.path.isfile(modelfile):
        print('UDPipe model not found. Downloading...', file=sys.stderr)
        wget.download(udpipe_model_url)

    print('\nLoading the model...', file=sys.stderr)
    model = Model.load(modelfile)
    process_pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')
    tagged_text = []
    print('Processing input...', file=sys.stderr)
    for line in text:
        # line = unify_sym(line.strip()) # здесь могла бы быть ваша функция очистки текста
        output = process(process_pipeline, text=line)
        tagged_text.append(' '.join(output))
        print(' '.join(output))
    return tagged_text

UDPipe model not found. Downloading...

Loading the model...
Processing input...


In [ ]:
tokens_sentences_tg = tag_ud(text_from_telegram)

In [37]:
tokens_sentences_tg[0:10]

['е_PROPN',
 'с_ADP',
 '',
 'бы_PART',
 'о_ADP',
 'река_NOUN',
 'быть_NOUN',
 'том_NOUN',
 'с_ADP',
 'я_PRON']

In [35]:
vectors = [model2.get_vector(word) for word in tokens_sentences_tg if word in list(model2.index_to_key)]

In [39]:
vectors[0]

array([ 3.75830047e-02,  2.56880037e-02,  2.57930029e-02,  8.45400058e-03,
        2.38290038e-02, -6.04270063e-02,  1.32200019e-02,  5.35640083e-02,
       -1.25220018e-02,  1.82400011e-02,  8.77640098e-02,  2.60150041e-02,
        1.44220022e-02, -2.56100018e-03,  5.92200086e-02,  6.62450045e-02,
        2.28660014e-02, -2.81800032e-02,  9.00000123e-06, -1.44130019e-02,
       -6.42990097e-02,  7.93600082e-03, -7.99100101e-03,  2.95840036e-02,
        2.55240034e-02, -6.46600081e-03, -4.95900027e-03,  6.70830086e-02,
        3.80420052e-02,  6.14570081e-02, -5.22960089e-02,  1.83880012e-02,
       -9.08900052e-03,  2.43640039e-02,  1.10286012e-01, -4.66810055e-02,
       -5.94990067e-02, -2.85750031e-02, -3.64390053e-02,  1.15590012e-02,
        4.22530025e-02, -4.29350026e-02, -2.62700040e-02,  5.97040057e-02,
       -4.29770052e-02,  3.92300030e-03, -3.18880044e-02, -5.77270091e-02,
       -4.30300049e-02,  1.64410025e-02,  7.58200064e-02,  3.66000040e-03,
        1.77540015e-02,  